# Problem Statement
To predict the ham/spam label based on message length and punctuations counts.

# Import modules 

In [2]:
import numpy as np
import pandas as pd


# Load a dataset

In [22]:
data = pd.read_csv('../TextFiles/smsspamcollection.tsv', sep='\t')
data.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


# Checking the missing values

In [4]:
data.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

# Count the no.of ham and spam labels

In [5]:
data['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

Total messages  are 5572 in which 4825 are ham and 747 are spam

# Split the dataset into train and test

In [6]:
from sklearn.model_selection import train_test_split

X= data['message']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Text preprocessing using CountVectorizer

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3733, 7082)

Here it shows training data is 3733 documents and 7082 features

# Transform Counts to Frequencies with Tf-idf

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3733, 7082)

here fit_transform is used to perform two task
- First it fits an estimator to the data
- Second transforms our count-matrix to a tf-idf representation

# Combining CountVectorizer and TfidTransformer steps to one 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) #use the original X_train set
X_train_tfidf.shape

(3733, 7082)

# Using LinearSVC Classifier

In [11]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf, y_train)

LinearSVC()

# Build a Pipeline

In [12]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])


text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [13]:
predictions = text_clf.predict(X_test)

In [14]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [15]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



# Final accuracy

In [16]:
print(metrics.accuracy_score(y_test,predictions))

0.989668297988037


Here our model performed well it correctly predicted spam 98.97% of the time!

Lets check our model perdictions if new data comes in

In [17]:
text_clf.predict(['Hello how are you'])

array(['ham'], dtype=object)

In [20]:
text_clf.predict(["You could be entitled up to £3,160 in compensation from mis-sold PPI on a credit card or loan. Please reply PPI for info or STOP to opt out."])

array(['spam'], dtype=object)

Conclusion our Model Performs well in Predictions of Spam and ham messages